# **Implementation of SpaRCe model using ReservoirPy library**

---



In [4]:
pip install reservoirpy

# Loading and scaling **MNIST dataset**

In [12]:
import numpy as np
from keras.datasets import mnist

In [13]:
def load_mnist_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    # Flatten the images to 28x28 and scale the values
    X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
    X_test = X_test.reshape(X_test.shape[0], -1) / 255.0
    return X_train, y_train, X_test, y_test

In [14]:
# Load and preprocess data
X_train, y_train, X_test, y_test = load_mnist_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


PyTorch

In [ ]:
# Import
import torch
from torchvision import datasets, transforms

In [15]:
def load_mnist_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.view(-1))
    ])

    train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='data', train=False, download=True, transform=transform)

    X_train, y_train = train_dataset.data.float() / 255.0, train_dataset.targets
    X_test, y_test = test_dataset.data.float() / 255.0, test_dataset.targets

    return X_train, y_train, X_test, y_test

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 12488825.21it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 347768.29it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:02<00:00, 659181.09it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 3412238.72it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
# Load the data
X_train, y_train, X_test, y_test = load_mnist_data()

# Creating a **sparse reservoir**

In [16]:
# Import
from reservoirpy.nodes import Reservoir

In [17]:
# Create a sparse reservoir
def create_sparse_reservoir(input_dim, res_size, density=0.1):
    # Create a sparse random weight matrix
    sparse_matrix = np.random.rand(res_size, res_size) < density
    weights = sparse_matrix.astype(float) * np.random.randn(res_size, res_size)
    # Initialize the reservoir with sparse connections
    reservoir = Reservoir(res_size, input_dim=input_dim, sr=0.9, W=weights)
    return reservoir

# **Readout layer with sparsity**

In [24]:
# Import
from reservoirpy.nodes import Ridge

In [25]:
# Read-out layer
def create_readout_layer(res_size, output_dim):
    readout = Ridge(res_size, output_dim)
    return readout


# **Training and evaluation of ESN**




In [20]:
# Import
from sklearn.metrics import accuracy_score

In [21]:
# Training and evaluation(accuracy) of ESN
def train_esn(reservoir, readout, X_train, y_train, X_test, y_test):
    # Apply the reservoir to the training data
    states_train = reservoir.run(X_train)
    states_test = reservoir.run(X_test)

    # Train the readout layer
    readout.fit(states_train, y_train)

    # Test the performance
    y_pred_train = readout.predict(states_train)
    y_pred_test = readout.predict(states_test)

    # Measure accuracy
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)

    print(f"Train Accuracy: {accuracy_train}")
    print(f"Test Accuracy: {accuracy_test}")
    return accuracy_train, accuracy_test


# Running the code

In [26]:
# Define parameters of SpaRCe ESN model
input_dim = X_train.shape[1]  # 28x28
reservoir_size = 1000
output_dim = 10
sparsity = 0.1

# Create the sparse ESN model
reservoir = create_sparse_reservoir(input_dim, reservoir_size, density=sparsity)
readout = create_readout_layer(reservoir_size, output_dim)

# Train and evaluate the model
accuracy_train, accuracy_test = train_esn(reservoir, readout, X_train, y_train, X_test, y_test)

TypeError: Data type '<class 'torch.Tensor'>' not understood. All vectors should be Numpy arrays in Reservoir-1.